In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import matplotlib.pyplot as plt
import os

# Set the environment variable to point to the license file on your Desktop
os.environ['GRB_LICENSE_FILE'] = os.path.expanduser('~/Desktop/gurobi.lic')


Loading data:

In [2]:
data = pd.read_csv('data_observed.csv')
data_observed = data[['X', 'Y', 'T', 'ips NN']]

In [27]:
def logistic(z):
    return 1.0 / (1.0 + np.exp(-z))

def solve_dual_lp(df_observed, policy, epsilon_0 = 0.1, epsilon_1 = 0.1, Gamma = 2):
    m = gp.Model("DualLP")
    m.setParam("OutputFlag", 0)  # Suppress solver output


    X_vals = df_observed["X"].values
    T_vals = df_observed["T"].values
    Y_vals = df_observed["Y"].values
    w_hat  = df_observed["ips NN"].values
    n = len(df_observed)
    I = range(n)


    I0 = [i for i in I if T_vals[i] == 0]
    I1 = [i for i in I if T_vals[i] == 1]
    Tset = [0, 1]


    
    def d(i, j):
        return (X_vals[i] - X_vals[j])**2
    

    eps = {0: epsilon_0, 1: epsilon_1}

    # Decision variables

    beta, gamma, theta, mu, nu = {}, {}, {}, {}, {}

    for t in Tset:
        beta[t] = m.addVar(lb=0.0, name=f"beta_{t}")

    for t in Tset:
        for i_ in I:
            gamma[(t, i_)] = m.addVar(lb = -GRB.INFINITY, name=f"gamma_{t}_{i_}")
    for t in Tset:
        for i_ in (I0 if t==0 else I1):
            theta[(t, i_)] = m.addVar(lb = -GRB.INFINITY, name=f"theta_{t}_{i_}")

    for i_ in I:
        mu[i_] = m.addVar(lb=0.0, name=f"mu_{i_}")
        nu[i_] = m.addVar(lb=0.0, name=f"nu_{i_}")

    # Objective
    obj_expr = gp.LinExpr()
    for t in Tset:
       obj_expr.addTerms(-eps[t], beta[t])
    for t in Tset:
        for i_ in I:
            obj_expr.addTerms(1.0 / n, gamma[(t, i_)])
    for i_ in I:
        obj_expr.addTerms((1.0 + (1.0 / Gamma)*(w_hat[i_] - 1.0)), mu[i_])
        obj_expr.addTerms(-(1.0 + Gamma*(w_hat[i_] - 1.0)), nu[i_])


    m.setObjective(obj_expr, GRB.MAXIMIZE)

    

    # Constraints
    for t in Tset:
        for i_ in (I0 if t == 0 else I1):
            lhs = gp.LinExpr()
            lhs += (1.0 / n) * policy[t,i_] * Y_vals[i_]
            lhs.addTerms(1 / (sum(w_hat)) , theta[(t, i_)])
            lhs.addTerms(-1.0, mu[i_])
            lhs.addTerms(1.0, nu[i_])
            m.addConstr(lhs >= 0, name=f"cA_t{t}_i{i_}")


    for t in Tset:
        for j_ in (I0 if t == 0 else I1):
            for i_ in I:
                lhs = gp.LinExpr()
                lhs.addTerms(d(i_, j_), beta[t])
                lhs.addTerms(-1.0, gamma[(t, i_)])
                lhs.addTerms(-1.0, theta[(t, j_)])
                m.addConstr(lhs >= 0, name=f"cB_t{t}_i{i_}_j{j_}")


    # Solve
    m.optimize()

    # Retrieve solution
    if m.status == GRB.OPTIMAL:
        return {
            "objective_value": m.objVal,
            "beta": {t: beta[t].X for t in Tset},
            "gamma": {(t, i_): gamma[(t, i_)].X for t in Tset for i_ in I},
            "theta": {(t, i_): theta[(t, i_)].X for t in Tset for i_ in (I0 if t==0 else I1)},
            "mu": {i_: mu[i_].X for i_ in I},
            "nu": {i_: nu[i_].X for i_ in I}
        }
    else:
        print(f"Model ended with non-OPTIMAL status: {m.status}")
        return None


In [26]:
# Define the range of Gamma values (for example, from 1 to 10)
Gamma_values = np.arange(1, 15)

# List to store the dual objective values for each Gamma
objective_values = []

# Define the sample policy (uniform 0.5 for all observations)
sample_policy = np.full((2, len(data_observed)), 0.5)

# Loop over the Gamma values
for Gamma in Gamma_values:
    print(f"Solving dual LP for Gamma = {Gamma}")
    sol = solve_dual_lp(data_observed, sample_policy, Gamma=Gamma)
    if sol is not None:
        obj_val = sol["objective_value"]
    else:
        obj_val = np.nan  # Changed from NAN to nan
    objective_values.append(obj_val)
    print(f"Gamma = {Gamma}, Objective Value = {obj_val}\n")

# Plot the objective values versus Gamma
plt.figure(figsize=(8, 5))
plt.plot(Gamma_values, objective_values, marker='o', linestyle='-')
plt.xlabel('Gamma')
plt.ylabel('Dual Objective Value')
plt.title('Dual Objective Value vs Gamma')
plt.grid(True)
plt.show()

Solving dual LP for Gamma = 1


KeyboardInterrupt: 

In [24]:
objective_values_df = pd.DataFrame({
    'Gamma': Gamma_values,
    'Objective_Value': objective_values
})

# Save to CSV
objective_values_df.to_csv('objective_values_dual.csv', index=False)